# chapter 6 Pandasを使ったデータ加工処理
# 6-1 概要と事前準備
## 6-1-1 この章で使用するライブラリのインポート

In [2]:
import numpy as np
import numpy.random as random
import scipy as sp
import pandas as pd
from pandas import Series, DataFrame

#　可視化ライブラリ
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

%precision 3

'%.3f'

In [3]:
# 6-2 Pandasの基本的なデータ操作
## 6-2-1 階層型インデックス

In [4]:
hier_df = DataFrame(
	np.arange(9).reshape((3,3)),
	index = [
		['a', 'a', 'b'],
		[1,2,2]
	],
	columns=[
		['Osaka', 'Tokyo', 'Osaka'],
		['Blue', 'Red', 'Red']
	]
)
hier_df

Osaka Tokyo Osaka
     Blue   Red   Red
a 1     0     1     2
  2     3     4     5
b 2     6     7     8